In [ ]:
import pandas as pd
import plotly.express as px
from oemof.solph import Bus, EnergySystem, Flow, Model
from oemof.solph.components import GenericStorage, Source, Sink, Transformer

from mtress._oemof_storage_multiplexer import storage_multiplexer_constraint

import graphviz

In [ ]:
# Define shapes for the component types
SHAPES = {Source: 'trapezium',
          Sink: 'invtrapezium',
          Bus: 'ellipse',
          Transformer: 'octagon',
          GenericStorage: 'cylinder'}


def generate_graph(energysystem):
    """Generate graphviz graph from energysystem."""
    dot = graphviz.Digraph(format='png')
    for node in energysystem.nodes:
        dot.node(node.label, shape=SHAPES.get(type(node), 'rectangle'))

    for node in energysystem.nodes:
        for output in node.outputs:
            dot.edge(node.label, output.label)

    return dot


In [ ]:
def get_flows(results):
    """
    Extract flows from results dictionary.

    To access the data you might want to use the xs function, i.e.
    >>> flows = get_flows(results)
    >>> flows.xs('component0', axis=1, level='source')
    >>> flows.xs('component0', axis=1, level='destination')

    :param results: Results from oemof optimization
    """
    flows = {
        (source_node.label, destination_node.label): result["sequences"]["flow"]
        for (source_node, destination_node), result in results.items()
        if destination_node is not None and not source_node == destination_node
    }

    return pd.concat(
        flows.values(), axis=1, names=["source", "destination"], keys=flows.keys()
    )


def get_status(results):
    """
    Extract status of flows from results dictionary.

    To access the data you might want to use the xs function, i.e.
    >>> flows = get_flows(results)
    >>> flows.xs('component0', axis=1, level='source')
    >>> flows.xs('component0', axis=1, level='destination')

    :param results: Results from oemof optimization
    """
    flows = {
        (source_node.label, destination_node.label): result["sequences"]["status"]
        for (source_node, destination_node), result in results.items()
        if destination_node is not None
        and "status" in result["sequences"]
        and not source_node == destination_node
    }

    return pd.concat(
        flows.values(), axis=1, names=["source", "destination"], keys=flows.keys()
    )


def get_variables(results):
    """
    Extract variables from results dictionary.

    To access the data you might want to use the xs function, i.e.
    >>> flows = get_flows(results)
    >>> flows.xs('component0', axis=1, level='component')
    >>> flows.xs('variable0', axis=1, level='variable_name')

    :param results: Results from oemof optimization
    """
    variables = {
        source_node.label
        if hasattr(source_node, "label")
        else str(source_node): result["sequences"]
        for (source_node, destination_node), result in results.items()
        if destination_node is None
    }

    return pd.concat(
        variables.values(),
        axis=1,
        names=[
            "component",
        ],
        keys=variables.keys(),
    )

def flatten_columns(frame, divider="_"):
    """Reduce multiindex columns to single level."""
    if frame.columns.nlevels <= 1:
        return frame

    return frame.set_axis(
        [divider.join(labels) for labels in frame.columns], axis=1, inplace=False
    )


In [ ]:
es = EnergySystem(timeindex=pd.date_range("2022-01-01", freq="30T", periods=12))

out_0 = Bus(label="out_0")
out_1 = Bus(label="out_1")
out_2 = Bus(label="out_2")

in_0 = Bus(label="in_0")
in_1 = Bus(label="in_1")
in_2 = Bus(label="in_2")

multiplexer = Bus(
    label="multiplexer",
    inputs={
        in_0: Flow(),
        in_1: Flow(),
        in_2: Flow(),
    },
    outputs={
        out_0: Flow(),
        out_1: Flow(),
        out_2: Flow(),
    },
)

storage = GenericStorage(
    label="storage",
    nominal_storage_capacity=2,
    initial_storage_level=0,
    balanced=True,
    inputs={multiplexer: Flow()},
    outputs={multiplexer: Flow()},
)

es.add(multiplexer, out_0, out_1, out_2, in_0, in_1, in_2, storage)

# slack_out = Bus(
#     label="slack_out",
#     outputs={
#         out_0: Flow(),
#         out_1: Flow(),
#         out_2: Flow(),
#     },
# )
# slack_out_source = Source(
#     label="slack_out_source",
#     outputs={slack_out: Flow(variable_costs=decreasing_cost)},
# )

# slack_in = Bus(
#     label="slack_in",
#     inputs={
#         in_0: Flow(),
#         in_1: Flow(),
#         in_2: Flow(),
#     },
# )
# slack_in_sink = Sink(
#     label="slack_in_sink",
#     inputs={slack_in: Flow(variable_costs=decreasing_cost)},
# )

# es.add(slack_out, slack_out_source, slack_in, slack_in_sink)

in_flow_ts = {
    in_0:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    in_1:  [0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0],
    in_2:  [0, 0, 2, 0, 0, 2, 2, 0, 0, 1, 0, 0],
}

out_flow_ts = {
    out_0: [0, 0, 0, 2, 1, 0, 0, 2, 1, 0, 1, 2],
    out_1: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    out_2: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
}

for b, ts in in_flow_ts.items():
    source = Source(
        label=f"source_{b.label}",
        outputs={b: Flow(nominal_value=1, fix=ts)},
    )
    es.add(source)

for b, ts in out_flow_ts.items():
    sink = Sink(
        label=f"sink_{b.label}",
        inputs={b: Flow(nominal_value=1, fix=ts)},
    )
    es.add(sink)


In [ ]:
# source = Source(
#     label=f"source_s",
#     outputs={multiplexer: Flow(nominal_value=1, fix=[1, 0, 0, 0, 0, 0])},
# )

# sink = Sink(
#     label=f"sink_s",
#     inputs={multiplexer: Flow(nominal_value=1, fix=[0, 0, 0, 1, 0, 0])},
# )

# es.add(source, sink)


In [ ]:
generate_graph(es)

In [ ]:

model = Model(es)


storage_multiplexer_constraint(
    model=model,
    name="multiplexer",
    storage_component=storage,
    multiplexer_component=multiplexer,
    input_level_components=[in_0, in_1, in_2],
    output_level_components=[out_0, out_1, out_2],
    levels=[0, 1, 2],
)

In [ ]:
model.solve(solver="cbc", solver_io="lp", solve_kwargs={"tee": True, "keepfiles": True})


In [ ]:
from oemof.solph.processing import results, meta_results

In [ ]:
my_results = results(model)


flows = get_flows(my_results)
variables = get_variables(my_results)

In [ ]:
px.line(flatten_columns(flows).join(flatten_columns(variables)))